# Plotting Summary
Sherry Wong, 9 June 2021


This notebook serves the purpose of consolidating all code from Fall 2020-2021.

### Import relevant packages:

In [1]:
import yt
from yt.units import kpc #for yt unit

import os #for file navigation
import os.path
from os import path

import shutil #remove excess files
import time #record runtime

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import ticker, cm #for log scale

import moviepy #for gif making
from moviepy.editor import ImageSequenceClip #for gif making
import beepy #sound for when the code is done running
import numpy as np #for working with numbers


## 1. Plotting in Matplotlib
### 1.1 Read/load files:

In [2]:
import h5py

#This method reads in a file, and creates a dictionary of relevant variables.
def fileInputTest(fileName):
    f2 = h5py.File(fileName, 'r')

    print("\nReading file:")
    print(f2) #general info
    print(f2.attrs)
    print("\nKeys:")
    print(f2.keys())
    print()

    coords = f2['coordinates']
    cray = f2['cray']
    dens = f2['dens']
    magx = f2['magx']
    magy = f2['magy']
    magz = f2['magz']
    pres = f2['pres']
    velx = f2['velx']
    vely = f2['vely']
    velz = f2['velz']
    temp = f2['temp']

    dict = {
        "coordinates" : coords,
        "cray_pressure" : cray,
        "density" : dens,
        "magx" : magx,
        "magy" : magy,
        "magz" : magz,
        "pressure" : pres,
        "velx" : velx,
        "vely" : vely,
        "velz" : velz,
        "temp" : temp

    }
    return dict
    #END OF METHOD

#Full file path. Jupyter likes Windows notation, Spyder likes Mac/Linux
filename = r"C:\Users\wongb\Documents\URS Data\m1.5_c1_16x16_128x128_Rodrigues_Streaming\More Plot Files\parkerCRs_hdf5_plt_cnt_0076"
#filename = "/mnt/c/Users/wongb/Documents/URS Data/m1.5_c1_16x16_128x128_Rodrigues_Streaming/More Plot Files/parkerCRs_hdf5_plt_cnt_0000"

dict = fileInputTest(filename)

#print(dict)
#print(dict['density'])

OSError: Unable to open file (unable to open file: name = 'C:\Users\wongb\Documents\URS Data\m1.5_c1_16x16_128x128_Rodrigues_Streaming\More Plot Files\parkerCRs_hdf5_plt_cnt_0076', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

### 1.2 Plotting with matplotlib:
#### Prelude:

The coordinate system for the FLASH simulation is a Z-order curve. It doesn't matter too much for our purposes, but it looks cool:

In [ ]:
#recreate the z-order curve:
posX = [] #list
posY = [] #list
for coord in dict['coordinates']:
    posX.append(coord[0])
    posY.append(coord[1])
plt.plot(posX, posY, linewidth = 1)
plt.margins(x=-0.4, y=-0.39) #zoom
#plt.savefig('CoordPlot1.png'); #bad quality
#plt.savefig('CoordPlot1.svg'); #much better quality
###print("\nPlotting finished.")

#### Expanding coordinate grid

<img src="Data_8x8.png" width="300" style="float: right; padding-left: 30px">

The way that the data is set up, each single coordinate represents a 8x8 box of additional data points. Thus, in order to view the full resolution, it is necessary to expand the each coordinate into an 8x8 grid, with the original coordinate in the bottom left corner. The other data will populate these 8x8 grids.


Without this extra work, plots will be much less resolved:

In [ ]:
#First convert the HDF5 data into lists
posX = [] #list
posY = [] #list
for coord in dict['coordinates']:
    posX.append(coord[0])
    posY.append(coord[1])
    
density = [] #list
for dens in dict['density']:
     #a random point in the 8x8
    density.append(dens[0][4][4])
    
fig = plt.figure(figsize=(6, 6))
plt.scatter(posX, posY,
           linewidths=0, alpha=.7,
           edgecolor='k',
           s = 5,
           c=density)

Lets do this properly. First, create a new coordinate system that includes the additional data points.

In [ ]:
"""Sets up coordinates for 16384 * 64 data points."""
posX = [] #list
posY = [] #list
#these steps may differ between simulations
stepX = dict['coordinates'][1][0] - dict['coordinates'][0][0]
stepY = dict['coordinates'][2][1] - dict['coordinates'][1][1]

for coord in dict['coordinates']:
    
    #equally space x and y apart
    tempXlin = np.linspace(coord[0], coord[0] + stepX, 8)
    tempYlin = np.linspace(coord[1], coord[1] + stepY, 8)
    
    #create a grid of coordinates using these XY values
    tempMeshgrid = np.meshgrid(tempXlin, tempYlin)
    
    #append the full X and Y coordinate pairs
    posX.append(tempMeshgrid[0])
    posY.append(tempMeshgrid[1])
    
posXarray, posYarray = (np.asarray(posX), np.asarray(posY))
posXarray = posXarray.flatten()
posYarray = posYarray.flatten()

Now let's set up the actual data. These don't require extra work; they are already expanded.

In [ ]:
"""Density Setup"""
density = [] #list
for dens in dict['density']:
    density.append(dens) #shape=(1, 8, 8)
densityArray = np.asarray(density)
densityArray = densityArray.flatten()

""""Temp Setup"""
temp = [] #list
for thing in dict['temp']:
    temp.append(thing) #shape=(1, 8, 8)
tempArray = np.asarray(temp)
tempArray = tempArray.flatten()

Now we are ready to plot our data.

In [ ]:
plt.tricontourf(posXarray, posYarray, densityArray, locator=ticker.LogLocator(), levels = 100) #good for irregular Z values
plt.colorbar(extend='both', orientation='vertical')

In [ ]:
z=tempArray
plt.title("Temp (\N{DEGREE SIGN}K)")
# plt.title("Density (g/$cm^3$)")

lev = np.logspace(np.log10(z.min()), np.log10(z.max()), num=1000)
plt.tricontourf(posXarray, posYarray, z, locator=ticker.LogLocator(), levels = lev) #good for irregular Z values
plt.colorbar(extend='both', orientation='vertical')